In [ ]:
import netCDF4
from distutils.util import strtobool
import matplotlib
import matplotlib.pyplot as plt
import sys
import os
import pdb
import platform
import numpy as np
import pyproj as pyproj
import datetime
import time
import bmi.wrapper
import pcrglobwb_bmi_v203 as pcrglobwb_bmi_v203
from pcrglobwb_bmi_v203 import pcrglobwb_bmi
from pcrglobwb_bmi_v203 import disclaimer
# from coupling_PCR_FM import coupling_functions
# from coupling_PCR_FM import model_functions
from coupling_PCR_FM import configuration
from coupling_PCR_FM import utils as utils

In [ ]:
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model

In [ ]:
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# -------------------------------------------------------------------------------------------------
# IMPORT MODEL SETTINGS FROM INI-FILE
# -------------------------------------------------------------------------------------------------

# parse set/ini-file with central/general settings for coupling framework
config = configuration.Configuration()
config.parse_configuration_file(argv1)

# parse env-file for user-specific paths and environmental variables
envs = configuration.Configuration()
envs.parse_configuration_file(argv2)

In [ ]:
# -------------------------------------------------------------------------------------------------
# SPECIFY MODEL SETTINGS
# -------------------------------------------------------------------------------------------------

type_hydrodynamicModel = config.hydrodynamic_model['model_name']
type_routingModel          = config.routing_model['model_name']
type_hydrologicModel      = config.hydrologic_model['model_name']

latlon = strtobool(config.general_settings['latlon'])
if latlon == False:
    inProj  = pyproj.Proj(init=config.hydrodynamic_model['model_projection'])

use_Fluxes = strtobool(config.general_settings['use_Fluxes'])
use_RFS = strtobool(config.general_settings['use_RFS'])
verbose = strtobool(config.general_settings['verbose'])

In [ ]:
# -------------------------------------------------------------------------------------------------
# SPECIFY NUMERICAL SETTINGS
# -------------------------------------------------------------------------------------------------

startTime_env						= config.numerical_settings['startTime']
endTime_env							= config.numerical_settings['endTime']
nr_timesteps 						  = utils.determineSteps(startTime_env, endTime_env)
update_step                           = int(config.numerical_settings['update_step'])

secPerDay                             = 86400.
end_time 							  = nr_timesteps * secPerDay
fraction_timestep 					  = secPerDay / update_step

threshold_inundated_depth             = float(config.numerical_settings['threshold_inundated_depth'])

# other
missing_value_landmask                = 255
missing_value_pcr                     = -999

In [ ]:
# -------------------------------------------------------------------------------------------------
# GET ENVIRONMENT SETTINGS
# -------------------------------------------------------------------------------------------------
mainFolder = os.getcwd()
inputDIR_env 	= envs.PCRpaths['inputDirectoryPCR']
outputDIR_env 	= envs.PCRpaths['outputDirectoryPCR']
DFM_path		= envs.DFM_engine['DFM_path'] #absolute
CMF_path		= os.path.join(mainFolder, envs.CMF_engine['CMF_path']) #relative

In [ ]:
# -------------------------------------------------------------------------------------------------
# SET PATHS TO MODELS
# -------------------------------------------------------------------------------------------------

# get main folder (i.e. GitHub folder)


# hydrodynamic model
hydrodynamicModel_dir       	= config.hydrodynamic_model['model_dir']
hydrodynamicModel_dir           = os.path.join(mainFolder, hydrodynamicModel_dir)
hydrodynamicModel_file      	= config.hydrodynamic_model['model_file']
hydrodynamicModel_proj		    = config.hydrodynamic_model['model_projection']

### ROUTING MODEL ###
routingModel_dir       			= config.routing_model['model_dir']
routingModel_dir           		= os.path.join(mainFolder, routingModel_dir)
routingModel_file      			= config.routing_model['model_file']
routingModel_file_tmp			= str('tmp_'+routingModel_file )
routingModel_config	   			= os.path.join(mainFolder, os.path.join(routingModel_dir, routingModel_file))
routingModel_config_tmp 		= os.path.join(mainFolder, os.path.join(routingModel_dir, routingModel_file_tmp))

# hydrologic model
hydrologicModel_dir				= config.hydrologic_model['config_dir']
hydrologicModel_file       		= config.hydrologic_model['config_file']
hydrologicModel_file_tmp		= str('tmp_'+hydrologicModel_file)
hydrologicModel_config	        = os.path.join(mainFolder, os.path.join(hydrologicModel_dir, hydrologicModel_file))
hydrologicModel_config_tmp 		= os.path.join(mainFolder, os.path.join(hydrologicModel_dir, hydrologicModel_file_tmp))

# overwriting inputDIR and outputDIR in PCR-GLOBWB ini-file with respect to personal environment
kwargs = dict(inputDir = inputDIR_env, outputDir = outputDIR_env)
utils.write_ini(hydrologicModel_config_tmp, hydrologicModel_config, **kwargs)

# parsing the overwritten PCR-GLOBWB ini-file
configPCR 						= configuration.Configuration()
configPCR.parse_configuration_file(hydrologicModel_config_tmp)

# retrieving informatino directly from PCR-GLOBWB ini-file
inputDIR 						= configPCR.globalOptions['inputDir']
clone_pcr 						= os.path.join(inputDIR, configPCR.globalOptions['cloneMap'])
landmask_pcr 					= os.path.join(inputDIR, configPCR.globalOptions['landmask'])

# initialize new

In [ ]:
# overwriting inputDIR and outputDIR in PCR-GLOBWB ini-file with respect to personal environment
kwargs = dict(inputDir = inputDIR_env, outputDir = outputDIR_env, startTime = startTime_env, endTime = endTime_env)
utils.write_ini(hydrologicModel_config_tmp, hydrologicModel_config, **kwargs)

hydrologicModel = PCR_model(hydrologicModel_config_tmp, missing_value_pcr)
# hydrologicModel.spinup()

In [ ]:
# overwriting nam-file
# kwargs = dict(CINPMAT = 'bla')
# utils.write_ini(routingModel_config_tmp, routingModel_config, **kwargs)

routingModel = CMF_model(CMF_path, os.path.join(routingModel_dir, routingModel_file), routingModel_dir)

In [ ]:
# coupling functions
def PCR2CMF_runoff(PCR_BMI):
    "creates a 'get_runoff' function to get runoff from PCR model fit for CMF model"
    def get_runoff():
        "get runoff from PCR model fit for CMF model"
        runoff = PCR_BMI.get_var('landSurfaceRunoff')
        runoff = np.where(np.isnan(runoff), 0, runoff)
        return runoff
    return get_runoff


CFM_get_runoff = PCR2CMF_runoff(hydrologicModel)
routingModel.set_options(get_runoff=CFM_get_runoff)

In [ ]:
for i in range(20):
    hydrologicModel.run()
    routingModel.run()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

runoff = hydrologicModel.get_var('landSurfaceRunoff')
outflw = routingModel.get_var('outflw')

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(runoff)
ax2.imshow(outflw)

In [ ]:
np.nanmax(runoff)